In [ ]:
!pip install transformers
!pip install -U datasets
!pip install torch
!pip install -U datasets fsspec huggingface_hub
!pip install pandas
!pip install nltk
!pip install tqdm
!pip install numpy
!pip install unbabel-comet

  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import pandas as pd
import torch, textwrap, time, re
from nltk.tokenize import word_tokenize
import nltk

from tqdm import tqdm
import numpy as np

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype="auto"
)

cot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
dataset = "poetry_dataset.csv"
try:
    df_local_csv = pd.read_csv(dataset)
    print(f"Dataset CSV '{dataset}' caricato con successo!")
except FileNotFoundError:
    print(f"Errore: Il file '{dataset}' non è stato trovato. Assicurati che sia nella stessa directory del tuo script.")
except Exception as e:
    print(f"Si è verificato un errore durante il caricamento del file CSV: {e}")

#print(dataset)
df_original = pd.read_csv("poetry_dataset.csv")
new_df = df_original[['poem_zh', 'poem_en']].rename(columns={
    'poem_zh': 'source',
    'poem_en': 'target'
})

print("Nuovo DataFrame con 'source' e 'target':")
print(new_df)


Dataset CSV 'poetry_dataset.csv' caricato con successo!
Nuovo DataFrame con 'source' e 'target':
                                               source  \
0   君生我未生，我生君已老。\n君恨我生迟，我恨君生早。\n君生我未生，我生君已老。\n恨不生同...   
1   幾行歸塞盡，暮雨相呼失。\n渚雲低暗渡，未必逢矰繳。\n\n片影獨何之，寒塘欲下遲。\n關月...   
2   昨夜见军贴，可汗大点兵。\n军书十二卷，卷卷有爷名。\n阿爷无大儿，木兰无长兄，\n愿为市鞍...   
3                          床前明月光，疑是地上霜。\n举头望明月，低头思故乡。   
4                  无边落木萧萧下，不尽长江滚滚来。\n万里悲秋常作客，百年多病独登台。   
5   怒发冲冠，凭栏处、潇潇雨歇。\n抬望眼、仰天长啸，壮怀激烈。\n三十功名尘与土，八千里路云和...   
6                                        蛇本无足，\n添画余恨。   
7                                        守株待兔，\n幸复再得。   
8                                       亡羊补牢，\n 未为晚矣。   
9                                        愚公移山，\n志坚可成。   
10                                       井底之蛙，\n不知天地。   
11                     红豆生南国,\n春来发几枝。\n愿君多采撷,\n此物最相思。   
12                                       月光思君,\n梦里相逢。   
13                                       心悦君兮,\n此情不移。   
14                                       君心似我,\n

In [ ]:
SYSTEM_TRANSLATOR_MSG = textwrap.dedent("""
    You are an expert translator translating from Chinese to English.
    Your task is to translate the given Chinese poem into English
    Think step by step before providing the final translation.

    Instructions:
    - Translate the given Chinese poem into English.
    - Please translate all lines of the poem, and keep the poetic structure.
    - Generate the full translation in a single FINAL output only after completing all your thoughts.
    - If you are still reasoning or analyzing the sentence, write exactly:
      Thought: <your thought or analysis>
    - Provide the Thoughts only in English.
    - After writing "FINAL:", continue writing the full translated poem in English without stopping.
    - Do not end the response right after writing "FINAL:"; provide the entire poem following it.


    Make sure the output always starts with "Thought:" or "FINAL:".
""")

In [ ]:
def one_step(messages, max_tokens=100, temperature=0.3):
    prompt = "".join(f"<|{m['role']}|>\n{m['content']}\n" for m in messages) + "<|assistant|>\n"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_new_tokens= max_tokens,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result[len(prompt):].strip().split("\n")[0]

In [ ]:
def chain_of_thought(user_query, max_steps=10, stream=False):
    messages = [
        {"role": "system", "content": SYSTEM_TRANSLATOR_MSG},
        {"role": "user", "content": user_query}
    ]
    scratchpad = []

    answer_lines = []
    collecting_final = False
    steps = 0

    while steps < max_steps:
        assistant_line = one_step(messages)
        steps += 1

        if stream:
            print(assistant_line)
            time.sleep(0.5)

        if assistant_line.startswith("FINAL:"):
            if not collecting_final:
                collecting_final = True
                final_line = assistant_line.removeprefix("FINAL:").strip()
                if final_line:
                    answer_lines.append(final_line)

        elif collecting_final:

            answer_lines.append(assistant_line)
        else:
            if not assistant_line.startswith("Thought:"):
                assistant_line = "Thought: " + assistant_line
            scratchpad.append(assistant_line)
            messages.append({"role": "assistant", "content": assistant_line})
            continue

        messages.append({"role": "assistant", "content": assistant_line})

    answer = "\n".join(answer_lines).strip()
    return scratchpad, answer


In [ ]:
from comet import download_model, load_from_checkpoint


comet_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)

def judge_translation(reference, hypothesis):
    """
    Evaluate the quality of a hypothetical translation against a reference using the COMET score.

    Args:
        reference (str): The reference sentence.
        hypothesis (str): The translated sentence to be evaluated.

    Returns:
        tuple: COMET score (float) and qualitative assessment (str).
    """

    data = [{
        "src": "",
        "mt": hypothesis,
        "ref": reference
    }]

    comet_score = comet_model.predict(data, batch_size=8, gpus=0)["scores"][0]

    if comet_score > 0.9:
        judgment = "Ottimo"
    elif comet_score > 0.6:
        judgment = "Buono"
    elif comet_score > 0.3:
        judgment = "Mediocre"
    else:
        judgment = "Scarso"

    return comet_score, judgment



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

checkpoints/model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
if __name__ == "__main__":
    for idx, row in new_df.iterrows():
        question = SYSTEM_TRANSLATOR_MSG + "\n\n" + row['source']

        print(f"\n=== ESEMPIO {idx+1} ===")
        thoughts, answer = chain_of_thought(question, stream=True, max_steps=20)

        print("TRADUZIONE ATTESA :", row['target'])

        comet, giudizio = judge_translation(row['target'], answer)
        print(f"Comet Score: {comet:.4f}")
        print("Giudizio del LLM Giudice:", giudizio)

    print("\n=== FINE ===")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



=== ESEMPIO 1 ===
Thought: Understanding the meaning of the poem seems to rely on the concept of reincarnation or the cycle of life and death. The first two lines suggest that the speaker regrets not being born at the same time as the person they love, while the third line emphasizes the age difference. The fourth line expresses a wish to be together since birth. The fifth and sixth lines seem to describe the situation where the speaker and the loved one are separated by great distances. The
Thought: last lines suggest a longing and a search for beauty, possibly in the form of flowers.
FINAL:
You were born when I wasn't,
I was born when you were old.
You regret my late birth,
I regret your early life.
You were born when I wasn't,
I was born when you were old.
Wishing we had been born at the same time,
Day after day, I long to be with you.
I was born when you weren't,
You were born when I was old.
Now we're separated by vast distances,
You're separated from me by the corner of the sea.

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


TRADUZIONE ATTESA : When you were born I was not yet born.
When I was born you were already mature.

What a pity for you that I was born late.
What a pity for me that you are now old.

When you were born I was not yet born.
When I was born you were already mature.

What a pity for us not to be born together.
How good it would be to live in your company.

When I was born you were not yet born.
When you were born I was a mature man.

What an immense distance separates me from you.
What a great ocean separates me from you.

When I was born you were not yet born.
When you were born I was a mature man.

I would like, as a butterfly, to fly on the flowers,
Sleep at night on the fragrant grass.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


...

Comet Score: 0.7099
Giudizio del LLM Giudice: Buono

=== ESEMPIO 26 ===
Thought: This line seems to describe a shallow strait or a body of water that evokes feelings of nostalgia or homesickness.
Thought: Shall I translate "乡愁" as "homesickness" or "nostalgia"? Both seem to fit. Let's go with "homesickness" for this translation.
Thought: The second line is: "人行不穷，水涯成碣."
Thought: This line seems to mean that there are countless people walking (or living), and the water's edge becomes a stone (or monument).
Thought: The third line is: "愁思长江夜，梦里漫漫长江流。"
Thought: This line describes the longing thoughts about the Long River (Longjiang) at night, and the dream where the Long River keeps flowing endlessly.
Thought: The fourth line is: "岁月如水，流不停。"
Thought: This line means that the passage of time is like flowing water, which never stops.
FINAL:
The first line is: "Homesickness is a shallow strait."
The second line is: "People never cease, water's edge becomes a monument."
The third line is: "N

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


TRADUZIONE ATTESA : Homesickness is a shallow strait.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s]

Comet Score: 0.5732
Giudizio del LLM Giudice: Mediocre

=== FINE ===


In [ ]:
def baseline_response(question, max_tokens=60):
    messages = [
        {"role": "system",
         "content": "You are a translator. Translate the following sentences directly from chinese to english,without going through other languages, and do so as concisely as possible."},
        {"role": "user", "content": question}
    ]
    return one_step(messages, max_tokens=max_tokens)

In [ ]:
def baseline_response(question, max_tokens=60):
    messages = [
        {"role": "system",
         "content": "You are a translator. Translate the following sentences from chinese to english, without going through other languages, and do so as concisely as possible."},
        {"role": "user", "content": "无边落木萧萧下，不尽长江滚滚来。万里悲秋常作客，百年多病独登台。"}

    ]

    return one_step(messages, max_tokens=max_tokens)
translation = baseline_response("无边落木萧萧下，不尽长江滚滚来。万里悲秋常作客，百年多病独登台。")

print(translation)

Beneath the boundless autumn leaves, the endless Long River flows on. For ten thousand miles in sorrow, I often stop by; for a hundred years I've borne my illness, alone ascending the stage.


In [ ]:
cot_comet_scores = []
noncot_comet_scores = []
subset_df = new_df.iloc[6:11]

if __name__ == "__main__":
    print("== INIZIO VALUTAZIONE ==")


    for idx, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc="Valutazione esempi"):
        question = SYSTEM_TRANSLATOR_MSG + "\n\n" + row['source']
        expected = row['target']

        #Chain of Thought (CoT)
        thoughts, cot_answer = chain_of_thought(question, stream=False, max_steps=5)
        comet_cot, _ = judge_translation(expected, cot_answer)
        cot_comet_scores.append(comet_cot)

        #Baseline
        noncot_answer = baseline_response(question)
        comet_noncot, _ = judge_translation(expected, noncot_answer)
        noncot_comet_scores.append(comet_noncot)

    avg_cot_comet = np.mean(cot_comet_scores)
    avg_noncot_comet = np.mean(noncot_comet_scores)

    print("\n== RISULTATI FINALI ==")
    print(f"Media COMET (CoT): {avg_cot_comet:.4f}")
    print(f"Media COMET (Non-CoT): {avg_noncot_comet:.4f}")

    if avg_cot_comet > avg_noncot_comet:
        print("Il modello CoT ha tradotto meglio in media.")
    elif avg_noncot_comet > avg_cot_comet:
        print("Il modello Non-CoT ha tradotto meglio in media.")
    else:
        print("Le prestazioni dei due modelli sono equivalenti.")

== INIZIO VALUTAZIONE ==


Valutazione esempi:   0%|          | 0/5 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_


== RISULTATI FINALI ==
Media BLEU (CoT): 0.6088
Media BLEU (Non-CoT): 0.5280
Il modello CoT ha tradotto meglio in media.
